In [3]:
################################### 12 题 ######################################
#    Implement Batch Gradient Descent with early stopping for Softmax Regression(without using SKLEARN)

In [77]:
## 思路：准备数据——转换one-hot——求训练集loss和梯度——求验证集loss，判断验证集loss是否不再下降——返回验证集最小loss，停止
from sklearn.datasets import load_iris
import numpy as np
iris = load_iris()
X = iris.data[:,[2,3]]
y = iris.target
N = len(y)
X_bias = np.c_[np.ones((N,1)), X]
test_ratio, val_ratio = 0.2, 0.2
np.random.seed(42)
shuffle_indeces = np.random.permutation(N)
len_val = np.int(N * val_ratio)
len_test = np.int(N * test_ratio)
X_test = X_bias[shuffle_indeces[:len_test]]
y_test = y[shuffle_indeces[:len_test]]
X_val = X_bias[shuffle_indeces[len_test:len_val+len_test]]
y_val = y[shuffle_indeces[len_test:len_val+len_test]]
X_train = X_bias[shuffle_indeces[len_val+len_test:]]
y_train = y[shuffle_indeces[len_val+len_test:]]

In [78]:
def to_one_hot(y):
    num_classes = len(np.unique(y)) #这样写其实不严格，比如数据较少时，len(np.unique(np.array([1,2,3,4,9]))) = 5
    sparse_y = np.zeros((len(y), num_classes))
    sparse_y[np.arange(len(y)),y] = 1   
    return sparse_y
y_train = to_one_hot(y_train)
y_val = to_one_hot(y_val)
y_test = to_one_hot(y_test)

In [87]:
X = np.random.randint(5, size=[3,3])
print(X)
print(X/np.sum(X,axis = 1, keepdims=True))
X/np.sum(X,axis = 1)


[[0 2 1]
 [3 2 0]
 [3 0 0]]
[[0.         0.66666667 0.33333333]
 [0.6        0.4        0.        ]
 [1.         0.         0.        ]]


array([[0.        , 0.4       , 0.33333333],
       [1.        , 0.4       , 0.        ],
       [1.        , 0.        , 0.        ]])

In [154]:
n_input = np.size(X_bias,1)
n_output = y.max()+1
theta = np.random.randn(n_input, n_output)
def softmax(scores):
    exp_scores = np.exp(scores)
    return exp_scores/np.sum(exp_scores,axis=1, keepdims=True)
#     return exp_scores/np.sum(exp_scores,axis=1) #错误写法：这样sum的结果不是一个列向量，而只是一个向量向量

In [155]:
# 注意： 这里加了early stopping之后，准确率无法达到百分之百，不加时可以达到。
# 当early stop 条件满足时，实际上取得不是最好的theta而是最好的theta之后一轮迭代的theta
## np.zeros, np.ones 加双括号！！
eta = 0.07
weight_decay = 0.1
EPSILON = 1e-10
best_loss = np.inf
EARLY_STOPPING = True
m = len(y_train)
for iter in range(5001):
    scores = np.dot(X_train, theta)
    preds = softmax(scores)
    error = preds - y_train
#     l2_loss = np.sum(np.square(theta))
    l2_reg = np.sum(np.square(theta[1:])) #正则化时不对w0进行惩罚,而theta的第一行对应每个二分类器的w0（对应X矩阵的第一列bias项）
    train_loss = -np.mean(np.sum(y_train * np.log(preds + EPSILON), axis = 1)) + weight_decay * l2_reg
    gradients = 1/m * X_train.T.dot(error) + weight_decay * np.r_[ np.zeros((1,n_output)), theta[1:]] #将λ/m合并在了λ里面？？？
    # 注意：由于l2_reg惩罚项不含w0，所以这里梯度的reg也要对去掉w0之外的w进行，但为了保持维度一致，可以在去除第一行之后的theta
    # 的第一行补上零
    theta = theta - eta * gradients
    
    preds_val = softmax(X_val.dot(theta))
    val_loss = -np.mean(np.sum(y_val * np.log(preds_val), axis = 1)) + weight_decay * l2_reg
    if iter %500 == 0:
        print('iter: {0} ,validation_loss: {1})'.format(iter, val_loss))
    if EARLY_STOPPING:
        if best_loss>val_loss:
            best_loss = val_loss
        else:
            print(iter-1, best_loss)
            print(iter, val_loss, 'early stopping!')
            break
    
    

iter: 0 ,validation_loss: 4.526865653040438)
iter: 500 ,validation_loss: 0.6069095202781684)
iter: 1000 ,validation_loss: 0.5834884313558759)
1307 0.5817246339823058
1308 0.5817246342698289 early stopping!


In [156]:
predict = np.argmax(softmax(X_val.dot(theta)),axis=1)
y_val_label  = np.argmax(y_val, axis=1)
accuracy = np.mean((predict==y_val_label))
print('accuracy: ', accuracy)

accuracy:  0.9666666666666667


In [153]:
predict

array([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 2, 0, 0, 1, 2, 2, 1, 2, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 1], dtype=int64)